In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import re
import os
import glob
import numpy as np
import pandas as pd
import cv2
from keras.preprocessing.image import load_img, save_img, img_to_array, array_to_img
import keras
print(keras.backend.image_data_format())
from keras.models import Model


In [ ]:
current_path = os.getcwd()
output_path = f"{current_path}/output"
csv_path = f"{output_path}/csv"
output_train_path = f"{output_path}/train"

In [ ]:
train_csv_list = glob.glob(f"{csv_path}/*_train.csv")


In [ ]:
pred_csv_list=train_csv_list.pop(1)
pred_csv_list = [pred_csv_list]
print([pred_csv_list])

In [ ]:
def read_csv_to_df(train_csv_list):
    columns = ["lu_x","lu_y","ru_x","ru_y","ld_x","ld_y","rd_x","rd_y"]
    return_dict = {}
    dir_list=[]
    df_concat = pd.Series([0,0,0,0,0,0,0,0],index = columns)
    for train_csv in train_csv_list:
        dir_path = train_csv.split('/')[-1].rstrip("_train.csv")
        print(train_csv)
        read_df = pd.read_csv(train_csv,index_col=0)
        df_concat = pd.concat([df_concat, read_df.T],axis=1)
        return_dict.update({dir_path:df_concat.T[1:]})
        dir_list.append(dir_path)
        df_concat = pd.Series([0,0,0,0,0,0,0,0],index = columns)
    return return_dict,dir_list

In [ ]:
train_csv_dict, train_dir_list = read_csv_to_df(train_csv_list)

#test_csv_dict, test_dir_list  = read_csv_to_df(test_csv_list)
pred_csv_dict, pred_dir_list  = read_csv_to_df(pred_csv_list)

label = train_csv_dict["001"].columns

In [ ]:
def data_pack(csv_dict, dir_list,train_path):
    img_shape = 200.
    x_data =[]
    y_data=[]
    data_list = []

    for name in dir_list:
        train_dir_path = f"{train_path}/{name}"

        index_list = csv_dict[name].index.values
        
        #tmp_dir = [f"{train_dir_path}/img_name" for img_name in index_list]
        data_list.extend(index_list)


        x_list = [img_to_array(load_img(f"{train_dir_path}/train_{val}.jpg"))/255. for val in index_list]
        
        y_list = [csv_dict[name].loc[val].values/img_shape for val in index_list]

        print(name)
        x_list = np.array(x_list)
        print(x_list.shape)
        y_list = np.array(y_list)
        print(y_list.shape)
        
        x_data.append(x_list)
        y_data.append(y_list)
        
        #return x_data, y_data
    return np.vstack(x_data), np.vstack(y_data),data_list

In [ ]:
x_train,y_train,train_data_list=data_pack(train_csv_dict, train_dir_list, output_train_path)
print("==============-")
pred_x_test,pred_y_test,pred_data_list=data_pack(pred_csv_dict, pred_dir_list,output_train_path)

In [ ]:
print(x_train.shape)

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.utils import plot_model

In [ ]:
model = Sequential()

print(x_train.shape[1:])
model.add(Conv2D(32, (5, 5), padding='same', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (1, 1), padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3) ,padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(256, (1, 1) ,padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, (1, 1), padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
#model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3), padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(256, (1, 1), padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
#model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(GlobalAveragePooling2D())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1]))
#model.add(Activation('softmax'))

In [ ]:
from keras.optimizers import Adam, Nadam
nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='mean_absolute_error',
                  optimizer=nadam,
                  metrics=['squared_hinge'])

In [ ]:
history = model.fit(x_train, y_train, batch_size=128, epochs=50, verbose=1, validation_split=0.1)

In [ ]:
model.save(f"{output_path}/test.h5")
#plot_model(model, to_file=f"{output_path}/test.png")

In [ ]:
from keras.models import load_model
l_mode=load_model(f"{output_path}/test.h5")

In [ ]:
score = l_mode.evaluate(pred_x_test, pred_y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
l_mode.summary()

In [ ]:
pred_his = l_mode.predict(pred_x_test)
print(pred_his)

In [ ]:
pred_his = l_mode.predict(pred_x_test[0:1])
print(pred_his)
print(pred_y_test[0:1])

In [ ]:
print(pred_x_test.shape)

In [ ]:
def check_img(image, y_label,name=None):
    im = image

    cv2.drawMarker(im,(int(y_label[0]),int(y_label[1])),(255,0,0), markerType=cv2.MARKER_STAR)
    cv2.drawMarker(im,(int(y_label[2]),int(y_label[3])),(0,255,0), markerType=cv2.MARKER_STAR)
    cv2.drawMarker(im,(int(y_label[4]),int(y_label[5])),(0,0,255), markerType=cv2.MARKER_STAR)
    cv2.drawMarker(im,(int(y_label[6]),int(y_label[7])),(0,0,0), markerType=cv2.MARKER_STAR)
    
    if name is not None:
        image_path =f"{current_path}/output/model/{name}.jpg"
        cv2.imwrite(image_path,im)
        return im
    else:
        return im
    del im

In [ ]:
#print((pred_x_test[0]*255).astype(np.int16))

In [ ]:
image=check_img((x_train[100]*255).astype(np.int16),y_train[100]*200)
plt.imshow(image)
test_img_path =f"{current_path}/output/model/test_img.jpg"
plt.savefig(test_img_path)

In [ ]:
for i,name in enumerate(train_data_list):
    x_img = (x_train[i]*255).astype(np.int16)
    pred_val = (y_train[i]*200).astype(np.int16)
    check_img(x_img,pred_val,name)

In [ ]:
for i,name in enumerate(pred_data_list):
    x_img = (pred_x_test[i]*255).astype(np.int16)
    pred_val = (pred_his[i]*200).astype(np.int16)
    check_img(x_img,pred_val,name)

In [ ]:
from keras.preprocessing import image
params = {
    'rotation_range': 20,
    'width_shift_range': 0.1
}
datagen = image.ImageDataGenerator(**params)
gen = datagen.flow(x, batch_size=16)
print(len(x_train))
model.fit_generator(gen, steps_per_epoch=len(x_train) / 32, epochs=16)